In [0]:
from pyspark.sql.functions import explode, sequence, to_date

In [0]:
spark.sql("""
        CREATE TABLE riders
        USING DELTA LOCATION '/delta/bronze_riders'
        """)

Out[4]: DataFrame[]

In [0]:
gold_riders = spark.sql("""
        SELECT rider_id,
            address,
            first as first_name,
            last as last_name,
            birthday,
            account_start_date,
            account_end_date,
            CASE is_member WHEN 'True' THEN 'Member'
                ELSE 'Casual Rider'
                END AS member_status
            FROM riders
        """)

In [0]:
gold_riders.show()

+--------+--------------------+-----------+---------+----------+------------------+----------------+-------------+
rider_id| address| first_name|last_name| birthday|account_start_date|account_end_date|member_status|
+--------+--------------------+-----------+---------+----------+------------------+----------------+-------------+
 1000| 1200 Alyssa Squares| Diana| Clark|1989-02-13| 2019-04-23| null| Member|
 1001| 397 Diana Ferry| Jennifer| Smith|1976-08-10| 2019-11-01| 2020-09-01| Member|
 1002|644 Brittany Row ...| Karen| Smith|1998-08-10| 2022-02-04| null| Member|
 1003|996 Dickerson Tur...| Bryan| Roberts|1999-03-29| 2019-08-26| null| Casual Rider|
 1004|7009 Nathan Expre...| Jesse|Middleton|1969-04-11| 2019-09-14| null| Member|
 1005|224 Washington Mi...| Christine|Rodriguez|1974-08-27| 2020-03-24| null| Casual Rider|
 1006| 1137 Angela Locks| Alicia| Taylor|2004-01-30| 2020-11-27| 2021-12-01| Member|
 1007| 979 Phillips Ways| Benjamin|Fernandez|1988-01-11| 2016-12-11| null| Casual Rider|
 1008| 7691 Evans Court| John| Crawford|1987-02-21| 2021-03-28| 2021-07-01| Member|
 1009|9922 Jim Crest Ap...| Victoria| Ritter|1981-02-07| 2020-06-12| 2021-11-01| Member|
 1010| 92973 Mary Ville| Tracy| Austin|1996-04-07| 2019-12-27| null| Member|
 1011|950 Grimes Burg A...| Jessica| Mcgee|1984-12-29| 2017-05-20| null| Member|
 1012|65532 Davis Sprin...| Heather| Fisher|1980-10-20| 2021-10-16| null| Member|
 1013| 7757 Johnston Roads| Timothy| Jones|1985-07-10| 2020-12-28| 2021-11-01| Member|
 1014| 501 Arellano Land| Jennifer| Martin|1989-12-04| 2017-11-24| null| Member|
 1015|3710 Rodriguez Gl...|Christopher| Silva|2001-07-25| 2017-07-10| null| Member|
 1016| 72226 Casey Square| Andrew| Jones|1991-12-13| 2022-02-02| null| Member|
 1017| 40395 Terrell Parks| William| Lawson|1981-04-17| 2019-03-11| null| Member|
 1018| 3514 Leslie Vista| Shelly| Briggs|1986-09-02| 2021-07-25| null| Casual Rider|
 1019|00348 Brandi Park...| Tina| Garcia|1997-05-03| 2021-07-10| 2022-01-01| Casual Rider|
+--------+--------------------+-----------+---------+----------+------------------+----------------+-------------+
only showing top 20 rows

In [0]:
gold_riders.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_riders')

In [0]:
spark.sql("""
        CREATE TABLE stations
        USING DELTA LOCATION '/delta/bronze_stations'
""")

Out[6]: DataFrame[]

In [0]:
gold_stations = spark.sql("""
                          SELECT station_id,
                          name AS station_name,
                          latitude,
                          longitude
                          FROM stations
                        """)

In [0]:
gold_stations.show()

+------------+--------------------+---------+----------+
 station_id| station_name| latitude| longitude|
+------------+--------------------+---------+----------+
 525|Glenwood Ave & To...| 42.0127| -87.66606|
KA1503000012| Clark St & Lake St|41.885796| -87.6311|
 637|Wood St & Chicago...|41.895634|-87.672066|
 13216| State St & 33rd St|41.834732|-87.625824|
 18003|Fairbanks St & Su...| 41.89581|-87.620255|
KP1705001026|LaSalle Dr & Huro...| 41.89488|-87.632324|
 13253|Lincoln Ave & Wav...|41.948795| -87.67528|
KA1503000044|Rush St & Hubbard St|41.890175| -87.62618|
KA1504000140|Winchester Ave & ...|41.924038|-87.676414|
TA1305000032|Clinton St & Madi...| 41.88224| -87.64107|
TA1306000012| Wells St & Huron St|41.894753| -87.6344|
 13133|Damen Ave & Cortl...| 41.91598| -87.67734|
 SL-005|Indiana Ave & Roo...| 41.86789| -87.62304|
 13235|Southport Ave & W...| 41.94815| -87.66394|
TA1307000139| MLK Jr Dr & 29th St|41.842052| -87.617|
TA1305000009|Clark St & Ida B ...| 41.87593|-87.630585|
 13276|Stockton Dr & Wri...|41.931347|-87.638695|
TA1307000107|Sheridan Rd & Mon...| 41.96167| -87.65464|
 13193|Larrabee St & Web...| 41.92182| -87.64414|
KA1503000072|Wacker Dr & Washi...|41.883133| -87.63732|
+------------+--------------------+---------+----------+
only showing top 20 rows

In [0]:
gold_stations.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_stations')

In [0]:
beginDate = '2013-01-31'
endDate = '2022-02-13'

(
  spark.sql(f"select explode(sequence(to_timestamp('{beginDate}'), to_timestamp('{endDate}'), interval 1 hour)) as calendarDateTime")
    .createOrReplaceTempView('datetimes')
)

In [0]:
gold_dates = spark.sql("""
    SELECT bigint(date_format(calendarDateTime, 'yyyyMMddHH')) AS datetime_key,
    calendarDateTime AS datetime_actual,
    int(date_format(calendarDateTime, 'yyyyMMdd')) AS date_key,
    to_date(calendarDateTime) AS date_actual,
    day(calendarDateTime) AS day,
    date_format(calendarDateTime, 'EEEE') AS day_name,
    hour(calendarDateTime) AS hour,
    CASE WHEN date_format(calendarDateTime, 'HH:mm') BETWEEN '05:00' and '08:29'
                             THEN 'Early Morning'
  	                        WHEN date_format(calendarDateTime, 'HH:mm') BETWEEN '08:30' and '11:59'
  		                      THEN 'Late Morning'
  	                        WHEN date_format(calendarDateTime, 'HH:mm') BETWEEN '12:00' and '17:59'
  		                      THEN 'Noon'
  	                        WHEN date_format(calendarDateTime, 'HH:mm') BETWEEN '18:00' and '22:29'
  		                      THEN 'Evening'
  	                        ELSE 'Night' END AS time_of_day,
     dayofweek(calendarDateTime) AS day_of_week,
     date_format(calendarDateTime, 'D') AS day_of_year,
     extract(week FROM calendarDateTime) AS week_of_year,
     date_format(calendarDateTime,'MMMM') AS month_name,
     extract(month FROM calendarDateTime) AS month,
     CASE date_format(calendarDateTime, 'Q') WHEN 1 THEN 'First' 
                      WHEN 2 THEN 'Second' WHEN 3 THEN 'Third' WHEN 4 THEN 'Fourth' END AS quarter_name,
     date_format(calendarDateTime, 'Q') AS quarter,
     extract(year FROM calendarDateTime) AS year
    FROM datetimes
""")

In [0]:
gold_dates.show()

+------------+-------------------+--------+-----------+---+--------+----+-------------+-----------+-----------+------------+----------+-----+------------+-------+----+
datetime_key| datetime_actual|date_key|date_actual|day|day_name|hour| time_of_day|day_of_week|day_of_year|week_of_year|month_name|month|quarter_name|quarter|year|
+------------+-------------------+--------+-----------+---+--------+----+-------------+-----------+-----------+------------+----------+-----+------------+-------+----+
 2013013100|2013-01-31 00:00:00|20130131| 2013-01-31| 31|Thursday| 0| Night| 5| 31| 5| January| 1| First| 1|2013|
 2013013101|2013-01-31 01:00:00|20130131| 2013-01-31| 31|Thursday| 1| Night| 5| 31| 5| January| 1| First| 1|2013|
 2013013102|2013-01-31 02:00:00|20130131| 2013-01-31| 31|Thursday| 2| Night| 5| 31| 5| January| 1| First| 1|2013|
 2013013103|2013-01-31 03:00:00|20130131| 2013-01-31| 31|Thursday| 3| Night| 5| 31| 5| January| 1| First| 1|2013|
 2013013104|2013-01-31 04:00:00|20130131| 2013-01-31| 31|Thursday| 4| Night| 5| 31| 5| January| 1| First| 1|2013|
 2013013105|2013-01-31 05:00:00|20130131| 2013-01-31| 31|Thursday| 5|Early Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013106|2013-01-31 06:00:00|20130131| 2013-01-31| 31|Thursday| 6|Early Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013107|2013-01-31 07:00:00|20130131| 2013-01-31| 31|Thursday| 7|Early Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013108|2013-01-31 08:00:00|20130131| 2013-01-31| 31|Thursday| 8|Early Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013109|2013-01-31 09:00:00|20130131| 2013-01-31| 31|Thursday| 9| Late Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013110|2013-01-31 10:00:00|20130131| 2013-01-31| 31|Thursday| 10| Late Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013111|2013-01-31 11:00:00|20130131| 2013-01-31| 31|Thursday| 11| Late Morning| 5| 31| 5| January| 1| First| 1|2013|
 2013013112|2013-01-31 12:00:00|20130131| 2013-01-31| 31|Thursday| 12| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013113|2013-01-31 13:00:00|20130131| 2013-01-31| 31|Thursday| 13| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013114|2013-01-31 14:00:00|20130131| 2013-01-31| 31|Thursday| 14| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013115|2013-01-31 15:00:00|20130131| 2013-01-31| 31|Thursday| 15| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013116|2013-01-31 16:00:00|20130131| 2013-01-31| 31|Thursday| 16| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013117|2013-01-31 17:00:00|20130131| 2013-01-31| 31|Thursday| 17| Noon| 5| 31| 5| January| 1| First| 1|2013|
 2013013118|2013-01-31 18:00:00|20130131| 2013-01-31| 31|Thursday| 18| Evening| 5| 31| 5| January| 1| First| 1|2013|
 2013013119|2013-01-31 19:00:00|20130131| 2013-01-31| 31|Thursday| 19| Evening| 5| 31| 5| January| 1| First| 1|2013|
+------------+-------------------+--------+-----------+---+--------+----+-------------+-----------+-----------+------------+----------+-----+------------+-------+----+
only showing top 20 rows

In [0]:
gold_dates.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_dates')

In [0]:
spark.sql("""
        CREATE TABLE trips
        USING DELTA LOCATION '/delta/bronze_trips'
        """)

Out[10]: DataFrame[]

In [0]:
spark.sql("""
        CREATE TABLE payments
        USING DELTA LOCATION '/delta/bronze_payments'
         """)

Out[11]: DataFrame[]

In [0]:
gold_fact_payment_details = spark.sql("""
          SELECT 
            payment_id AS fact_payment_details_key,
            dim_riders.rider_id AS rider_id,
            bigint(date_format(payments.date, 'yyyyMMddHH')) AS paymentdate_key,
            payments.amount as payment_amount
            FROM payments
            LEFT JOIN dim_riders
            ON payments.rider_id = dim_riders.rider_id;
         """)

In [0]:
gold_fact_payment_details.show()

+------------------------+--------+---------------+--------------+
fact_payment_details_key|rider_id|paymentdate_key|payment_amount|
+------------------------+--------+---------------+--------------+
 1| 1000| 2019050100| 9.00|
 2| 1000| 2019060100| 9.00|
 3| 1000| 2019070100| 9.00|
 4| 1000| 2019080100| 9.00|
 5| 1000| 2019090100| 9.00|
 6| 1000| 2019100100| 9.00|
 7| 1000| 2019110100| 9.00|
 8| 1000| 2019120100| 9.00|
 9| 1000| 2020010100| 9.00|
 10| 1000| 2020020100| 9.00|
 11| 1000| 2020030100| 9.00|
 12| 1000| 2020040100| 9.00|
 13| 1000| 2020050100| 9.00|
 14| 1000| 2020060100| 9.00|
 15| 1000| 2020070100| 9.00|
 16| 1000| 2020080100| 9.00|
 17| 1000| 2020090100| 9.00|
 18| 1000| 2020100100| 9.00|
 19| 1000| 2020110100| 9.00|
 20| 1000| 2020120100| 9.00|
+------------------------+--------+---------------+--------------+
only showing top 20 rows

In [0]:
gold_fact_payment_details.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('fact_payment_details')

In [0]:
gold_fact_trip_rider_details = spark.sql("""
          SELECT 
            CONCAT(trip_id, CAST(dim_riders.rider_id AS varchar(10))) AS trip_rider_key,
            trips.start_station_id AS start_station_id,
            trips.end_station_id AS end_station_id,
            dim_riders.rider_id AS rider_id,
            BIGINT(date_format(trips.start_at, 'yyyyMMddHH')) AS trip_start_key,
            INT(months_between(trips.start_at, dim_riders.birthday) / 12) AS rider_age_at_trip_years,
            INT(months_between(dim_riders.account_start_date, dim_riders.birthday) / 12) AS rider_age_at_account_start_years,
            BIGINT(trips.ended_at) - BIGINT(trips.start_at)  AS trip_duration_seconds
            FROM trips
            LEFT JOIN dim_riders
            ON trips.rider_id = dim_riders.rider_id;
         """)

In [0]:
gold_fact_trip_rider_details.show()

+--------------------+----------------+--------------+--------+--------------+-----------------------+--------------------------------+---------------------+
 trip_rider_key|start_station_id|end_station_id|rider_id|trip_start_key|rider_age_at_trip_years|rider_age_at_account_start_years|trip_duration_seconds|
+--------------------+----------------+--------------+--------+--------------+-----------------------+--------------------------------+---------------------+
89E7AA6C29227EFF7...| 525| 660| 71934| 2021021216| 37| 37| 407|
0FEFDE26035683654...| 525| 16806| 47854| 2021021417| 38| 37| 1171|
E6159D746B2DBB917...| KA1503000012| TA1305000029| 70870| 2021020919| 33| 32| 532|
B32D3199F1C2E75B5...| 637| TA1305000034| 58974| 2021020217| 19| 19| 265|
83E463F23575F4BF3...| 13216| TA1309000055| 39608| 2021022315| 71| 71| 914|
BDAA7E3494E8D5453...| 18003| KP1705001026| 36267| 2021022415| 27| 27| 332|
A7727423511712575...| KP1705001026| KP1705001026| 50104| 2021020117| 33| 29| 51|
295476889D9B79F81...| 18003| 18003| 19618| 2021021118| 23| 21| 76|
362087194BA4CC9A1...| KP1705001026| KP1705001026| 16732| 2021022715| 19| 15| 1377|
21630F715038CCB05...| KP1705001026| KP1705001026| 57068| 2021022008| 47| 45| 1042|
A977EB7FE7F5CD3A3...| KP1705001026| KP1705001026| 32712| 2021022008| 17| 15| 25|
8B868B03D6753C2A2...| KP1705001026| KP1705001026| 23227| 2021022016| 26| 25| 876|
BD331D658B9D2C317...| 525| 520| 73221| 2021021813| 15| 15| 257|
8DFEA9BAFE6BAA622...| 13253| TA1309000050| 22163| 2021022617| 42| 42| 164|
27BE9F6E67AFD86C7566| 525| 15578| 7566| 2021020614| 21| 21| 925|
9B790D47A0A0F7F17...| KA1503000044| KA1504000142| 71588| 2021021923| 19| 17| 2660|
3C2DF72600B1DE6C3...| KA1503000044| KA1504000142| 38661| 2021021823| 34| 32| 2489|
48A8D07ED9C7065C6...| KA1503000044| KA1504000142| 64751| 2021022023| 23| 23| 2509|
BBFF2AAA0A3A1A261...| KA1504000140| 17660| 10721| 2021020215| 25| 25| 888|
030723CBA8CF05E71...| TA1305000032| 15542| 13281| 2021022307| 21| 22| 285|
+--------------------+----------------+--------------+--------+--------------+-----------------------+--------------------------------+---------------------+
only showing top 20 rows

In [0]:
gold_fact_trip_rider_details.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('fact_trip_rider_details')